# TASK 1 - Analysis of Clinical Trial Data Using Spark SQL

Project Overview:
This project focuses on analyzing clinical trial data using Spark SQL. The goal was to clean, process, and explore key aspects of the dataset, such as study types, conditions, and trial durations.

Key steps:

1. Data Cleaning: Handled missing values and split multi-condition fields.

2. Exploratory Data Analysis (EDA): Investigated study types and conditions.

3. Key Analysis:

- Identified the most frequent study types and conditions.

- Calculated the average trial duration.

- Analyzed trends in diabetes-related clinical trials over time.

The analysis provided valuable insights into clinical trials, helping build a foundation for further research.

#### 1.0 Checking and ensuring the location after uploading the CSV file

The command dbutils.fs.ls("/FileStore/tables") in Databricks helps you see a list of all the files stored in the "tables" folder. It's like checking the contents of a specific folder to find your data files, like CSV or JSON, that you can use for your analysis. It’s a simple way to know what files are available for work in Databricks.

In [0]:
 dbutils.fs.ls("/FileStore/tables")

Out[220]: [FileInfo(path='dbfs:/FileStore/tables/Clinicaltrial_16012025.csv', name='Clinicaltrial_16012025.csv', size=205522181, modificationTime=1742335744000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1740590400000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1739632416000),
 FileInfo(path='dbfs:/FileStore/tables/emails.csv', name='emails.csv', size=1426122219, modificationTime=1739827992000),
 FileInfo(path='dbfs:/FileStore/tables/iotstream/', name='iotstream/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/Fil

#### 1.1 Creating the DataFrame

This code loads a CSV file named Clinicaltrial_16012025.csv into a DataFrame in Databricks. It reads the file from the /FileStore/tables/ location, treats the first row as column headers, automatically detects the data types, and handles special characters like quotes. The df.show(20) command displays the first 20 rows of the data for you to check if everything is loaded correctly.

In [0]:
df = spark.read.csv(
    "/FileStore/tables/Clinicaltrial_16012025.csv", 
    header=True, 
    inferSchema=True, 
    quote='"', 
    escape='"', 
    multiLine=True
)
df.show(20)

+-----------+--------------------+------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------------+--------------------+-------------------+-------------------+
| NCT Number|         Study Title|     Acronym|      Study Status|          Conditions|       Interventions|             Sponsor|       Collaborators|Enrollment|Funder Type|    Study Type|        Study Design|         Start Date|    Completion Date|
+-----------+--------------------+------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------------+--------------------+-------------------+-------------------+
|NCT05013879|Kinesiotape for E...|        null|         COMPLETED|Arthroplasty Comp...|DEVICE: Kinesio(R...|Montefiore Medica...|Burke Rehabilitat...|        65|      OTHER|INTERVENTIONAL|Allocation: RANDO...|2021-10-18 00:00:00|2023-11-24 00:00:00|


#### 1.3 Converting created Dataframe to Temporary View

This code creates  a temporary view called clinical_trials using the DataFrame df. A temporary view allows you to run SQL queries on the DataFrame just like you would with a table in a database. The view only exists for the duration of the session and is useful for running SQL operations on the data without altering the original DataFrame.

In [0]:
df.createOrReplaceTempView("clinical_trials")


#### 2.0 EDA

In [0]:
%sql
-- 1. Checking schema and columns
DESCRIBE clinical_trials;


col_name,data_type,comment
NCT Number,string,null
Study Title,string,null
Acronym,string,null
Study Status,string,null
Conditions,string,null
Interventions,string,null
Sponsor,string,null
Collaborators,string,null
Enrollment,int,null
Funder Type,string,null


####2.1 Checking for the Null Values


This SQL query is designed to check for NULL values in some key columns (Study Type, Conditions, Study Status, Start Date, Completion Date) of the clinical_trials view. It uses a CASE statement to count how many NULL values are present in each of these columns. The SUM function adds up the 1's returned by the CASE statements for each column, giving you the total number of NULL values for each field.

In simpler terms, it helps identify how many records are missing data for each of these important fields in the clinical trials dataset.

In [0]:
%sql
-- 3. Checking for NULLs in important columns
SELECT 
    SUM(CASE WHEN `Study Type` IS NULL THEN 1 ELSE 0 END) AS null_study_type,
    SUM(CASE WHEN `Conditions` IS NULL THEN 1 ELSE 0 END) AS null_conditions,
    SUM(CASE WHEN `Study Status` IS NULL THEN 1 ELSE 0 END) AS null_study_status,
    SUM(CASE WHEN `Start Date` IS NULL THEN 1 ELSE 0 END) AS null_start_date,
    SUM(CASE WHEN `Completion Date` IS NULL THEN 1 ELSE 0 END) AS null_completion_date
FROM clinical_trials;

null_study_type,null_conditions,null_study_status,null_start_date,null_completion_date
900,935,0,5187,16671


####2.2 Checking the unique Values

This SQL query retrieves all the unique or distinct values present in the Study Status column of the clinical_trials dataset. The DISTINCT keyword ensures that duplicate values are removed, so you only get one instance of each unique status.

In [0]:
%sql
-- 4. Geting distinct values for `Study Status` 
SELECT DISTINCT `Study Status` FROM clinical_trials;

Study Status
SUSPENDED
APPROVED_FOR_MARKETING
ENROLLING_BY_INVITATION
AVAILABLE
NO_LONGER_AVAILABLE
UNKNOWN
TERMINATED
NOT_YET_RECRUITING
TEMPORARILY_NOT_AVAILABLE
RECRUITING


In [0]:
%sql
SELECT * FROM clinical_trials
LIMIT 10;

NCT Number,Study Title,Acronym,Study Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Study Type,Study Design,Start Date,Completion Date
NCT05013879,Kinesiotape for Edema After Bilateral Total Knee Arthroplasty,null,COMPLETED,"Arthroplasty Complications|Arthroplasty, Replacement, Knee",DEVICE: Kinesio(R)Tape for edema control,Montefiore Medical Center,Burke Rehabilitation Hospital,65,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT,2021-10-18T00:00:00.000+0000,2023-11-24T00:00:00.000+0000
NCT00517179,Effect of Vardenafil on Blood Pressure in Patients With Erectile Dysfunction Who Received Concomitant Doxazosin GITS,null,COMPLETED,Prostatic Hyperplasia|Impotence,DRUG: Vardenafil 10mg,"Hospital Authority, Hong Kong",null,40,OTHER_GOV,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CROSSOVER|Masking: DOUBLE|Primary Purpose: TREATMENT,2006-04-01T00:00:00.000+0000,2007-05-01T00:00:00.000+0000
NCT06714279,Laparoscopic-Assisted Transversus Abdominus Plane Block Versus Intraperitoneal Irrigation of Local Anesthetic for Patients Undergoing Laparoscopic Cholecystectomy,null,NOT_YET_RECRUITING,Laparoscopic Cholecystectomy|TAP Block|Local Anesthetic,DRUG: Tap Block - Bupivacaine|DRUG: Intraperitoneal infiltration to liver,"Royal College of Surgeons, Ireland",null,144,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: NONE|Primary Purpose: TREATMENT,2025-01-01T00:00:00.000+0000,2025-01-01T00:00:00.000+0000
NCT05600179,OCTA in Epivascular Glia After Dex Implant,null,COMPLETED,Diabetic Retinopathy,DRUG: Dexamethasone intravitreal implant,Federico II University,null,38,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-01-01T00:00:00.000+0000,2022-09-30T00:00:00.000+0000
NCT01511679,Brain-imaging and Adolescent Neuroscience Consortium,BANC,WITHDRAWN,Alcohol Abuse,null,Boston Children's Hospital,Massachusetts General Hospital|Mclean Hospital|Massachusetts Institute of Technology,0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2012-09-01T00:00:00.000+0000,2017-09-01T00:00:00.000+0000
NCT05602779,Leverage Noninvasive Transcutaneous Vagus Nerve Stimulation to Reduce Suicidal Behaviors in Vulnerable Adolescents,null,RECRUITING,Self Harm|Suicidal Ideation,DEVICE: tVns Program|OTHER: Phone App Program|COMBINATION_PRODUCT: tVNS and Phone App Program|OTHER: Enhanced Treatment as Usual,University of Notre Dame,University of Rochester,212,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE (PARTICIPANT)|Primary Purpose: PREVENTION,2023-10-08T00:00:00.000+0000,2027-09-30T00:00:00.000+0000
NCT04175379,The Effect of Permissive Hypercapnia on Oxygenation and Post-operative Pulmonary Complication During One-lung Ventilation,null,UNKNOWN,Thoracic Surgery,OTHER: group 40|OTHER: group 50|OTHER: group 60,Yonsei University,null,279,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: TRIPLE (PARTICIPANT, CARE_PROVIDER, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",2019-11-25T00:00:00.000+0000,2021-10-01T00:00:00.000+0000
NCT01126879,Genistein in Treating Patients With Prostate Cancer,null,TERMINATED,Adenocarcinoma of the Prostate|Recurrent Prostate Cancer|Stage I Prostate Cancer|Stage II Prostate Cancer|Stage III Prostate Cancer,DIETARY_SUPPLEMENT: genistein|OTHER: placebo|PROCEDURE: therapeutic conventional surgery,Northwestern University,National Cancer Institute (NCI),12,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (PARTICIPANT, INVESTIGATOR)|Primary Purpose: TREATMENT",2011-02-03T00:00:00.000+0000,2013-12-28T00:00:00.000+0000
NCT03058679,Trial of Specific Carbohydrate and Mediterranean Diets to Induce Remission of Crohn's Disease,DINE-CD,COMPLETED,Crohn Disease,OTHER: Diet,University of Pennsylvania,"Patient-Centered Outcomes Research Institute|Crohn's and Colitis Foundation|Univer

####2.3 Replacing the Null Value with the string " NULL VALUES"

The SQL query creates a temporary view named cleaned_study_type from the clinical_trials dataset. It keeps all the original data while addressing any missing values in the Study Type column by replacing them with the text 'NULL VALUES'. This approach ensures that any missing data is clearly labeled, making it easier to identify and handle gaps in the dataset without having to deal with NULLs directly during analysis.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cleaned_study_type AS
SELECT *,
       -- Replacing NULL with 'NULL VALUES'
       IFNULL(`Study Type`, 'NULL VALUES') AS study_type_fixed
FROM clinical_trials;


In [0]:
%sql
SELECT * FROM cleaned_study_type
LIMIT 10;

NCT Number,Study Title,Acronym,Study Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Study Type,Study Design,Start Date,Completion Date,study_type_fixed
NCT05013879,Kinesiotape for Edema After Bilateral Total Knee Arthroplasty,null,COMPLETED,"Arthroplasty Complications|Arthroplasty, Replacement, Knee",DEVICE: Kinesio(R)Tape for edema control,Montefiore Medical Center,Burke Rehabilitation Hospital,65,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT,2021-10-18T00:00:00.000+0000,2023-11-24T00:00:00.000+0000,INTERVENTIONAL
NCT00517179,Effect of Vardenafil on Blood Pressure in Patients With Erectile Dysfunction Who Received Concomitant Doxazosin GITS,null,COMPLETED,Prostatic Hyperplasia|Impotence,DRUG: Vardenafil 10mg,"Hospital Authority, Hong Kong",null,40,OTHER_GOV,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CROSSOVER|Masking: DOUBLE|Primary Purpose: TREATMENT,2006-04-01T00:00:00.000+0000,2007-05-01T00:00:00.000+0000,INTERVENTIONAL
NCT06714279,Laparoscopic-Assisted Transversus Abdominus Plane Block Versus Intraperitoneal Irrigation of Local Anesthetic for Patients Undergoing Laparoscopic Cholecystectomy,null,NOT_YET_RECRUITING,Laparoscopic Cholecystectomy|TAP Block|Local Anesthetic,DRUG: Tap Block - Bupivacaine|DRUG: Intraperitoneal infiltration to liver,"Royal College of Surgeons, Ireland",null,144,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: NONE|Primary Purpose: TREATMENT,2025-01-01T00:00:00.000+0000,2025-01-01T00:00:00.000+0000,INTERVENTIONAL
NCT05600179,OCTA in Epivascular Glia After Dex Implant,null,COMPLETED,Diabetic Retinopathy,DRUG: Dexamethasone intravitreal implant,Federico II University,null,38,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-01-01T00:00:00.000+0000,2022-09-30T00:00:00.000+0000,OBSERVATIONAL
NCT01511679,Brain-imaging and Adolescent Neuroscience Consortium,BANC,WITHDRAWN,Alcohol Abuse,null,Boston Children's Hospital,Massachusetts General Hospital|Mclean Hospital|Massachusetts Institute of Technology,0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2012-09-01T00:00:00.000+0000,2017-09-01T00:00:00.000+0000,OBSERVATIONAL
NCT05602779,Leverage Noninvasive Transcutaneous Vagus Nerve Stimulation to Reduce Suicidal Behaviors in Vulnerable Adolescents,null,RECRUITING,Self Harm|Suicidal Ideation,DEVICE: tVns Program|OTHER: Phone App Program|COMBINATION_PRODUCT: tVNS and Phone App Program|OTHER: Enhanced Treatment as Usual,University of Notre Dame,University of Rochester,212,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE (PARTICIPANT)|Primary Purpose: PREVENTION,2023-10-08T00:00:00.000+0000,2027-09-30T00:00:00.000+0000,INTERVENTIONAL
NCT04175379,The Effect of Permissive Hypercapnia on Oxygenation and Post-operative Pulmonary Complication During One-lung Ventilation,null,UNKNOWN,Thoracic Surgery,OTHER: group 40|OTHER: group 50|OTHER: group 60,Yonsei University,null,279,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: TRIPLE (PARTICIPANT, CARE_PROVIDER, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",2019-11-25T00:00:00.000+0000,2021-10-01T00:00:00.000+0000,INTERVENTIONAL
NCT01126879,Genistein in Treating Patients With Prostate Cancer,null,TERMINATED,Adenocarcinoma of the Prostate|Recurrent Prostate Cancer|Stage I Prostate Cancer|Stage II Prostate Cancer|Stage III Prostate Cancer,DIETARY_SUPPLEMENT: genistein|OTHER: placebo|PROCEDURE: therapeutic conventional surgery,Northwestern University,National Cancer Institute (NCI),12,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (PARTICIPANT, INVESTIGATOR)|Primary Purpose: TREATMENT",2011-02-03T00:00:00.000+0000,2013-12-28T00:00:00.000+0000,INTERVENTIONAL
NCT03058679,Trial of Specific Carbohydrate and Mediterranean Diets to Induce Remission of Crohn's Disease,DINE-CD,COMPLETED,C

###3.0 Question 1 : Listing all the clinical trial types (as contained in the Type column of the data) along with their frequency, sorting the results from most to least frequent

This SQL query retrieves the frequency count of each unique value in the study_type_fixed column from the cleaned_study_type temporary view. It groups the data by the study_type_fixed column, counting how many times each value appears. The results are then ordered by the frequency in descending order, so the most common values will appear first. Essentially, this helps identify how frequently each study type (including the newly added "NULL VALUES") occurs in the dataset.

In [0]:
%sql
SELECT 
    study_type_fixed, 
    COUNT(*) AS frequency
FROM 
    cleaned_study_type
GROUP BY 
    study_type_fixed
ORDER BY 
    frequency DESC;


study_type_fixed,frequency
INTERVENTIONAL,399888
OBSERVATIONAL,120906
EXPANDED_ACCESS,966
NULL VALUES,900


This SQL query performs a similar task as the previous one but with an added condition. It retrieves the frequency count of each unique value in the study_type_fixed column, but it excludes the rows where study_type_fixed has the value "NULL VALUES". The data is grouped by the study_type_fixed column, and the count of each value is calculated. Finally, the results are ordered in descending order of frequency, ensuring that the most common study types (excluding "NULL VALUES") appear at the top. This helps in understanding the distribution of actual study types while ignoring missing or null values.

In [0]:
%sql
SELECT 
    study_type_fixed, 
     COUNT(*) AS frequency
FROM 
    cleaned_study_type
WHERE 
    study_type_fixed  != 'NULL VALUES'
GROUP BY 
    study_type_fixed
ORDER BY 
    frequency DESC;


study_type_fixed,frequency
INTERVENTIONAL,399888
OBSERVATIONAL,120906
EXPANDED_ACCESS,966


In [0]:
%sql
SELECT 
    Conditions
FROM
    cleaned_study_type

Conditions
"Arthroplasty Complications|Arthroplasty, Replacement, Knee"
Prostatic Hyperplasia|Impotence
Laparoscopic Cholecystectomy|TAP Block|Local Anesthetic
Diabetic Retinopathy
Alcohol Abuse
Self Harm|Suicidal Ideation
Thoracic Surgery
Adenocarcinoma of the Prostate|Recurrent Prostate Cancer|Stage I Prostate Cancer|Stage II Prostate Cancer|Stage III Prostate Cancer
Crohn Disease
Severe Aplastic Anemia


####4.0 Creating Temporary for seperating and spliting Conditions 


This SQL statement creates or updates a temporary view called cleaned_conditions. It processes the Conditions column from the clinical_trials dataset by splitting each entry using the pipe symbol (|) as a delimiter—this is useful when multiple conditions are listed in a single row. Then, it uses explode to turn each condition into its own separate row. The result is a cleaner, more analysis-friendly view where each row contains only one medical condition, making it easier to count or analyze individual conditions.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cleaned_conditions AS
SELECT
    explode(split(Conditions, '\\|')) AS condition
FROM 
    clinical_trials;



In [0]:
%sql
SELECT 
    condition
FROM
    cleaned_conditions
    LIMIT 30

condition
Arthroplasty Complications
"Arthroplasty, Replacement, Knee"
Prostatic Hyperplasia
Impotence
Laparoscopic Cholecystectomy
TAP Block
Local Anesthetic
Diabetic Retinopathy
Alcohol Abuse
Self Harm


In [0]:
%sql
SELECT 
    count(condition) AS count_of_condition 
FROM
    cleaned_conditions

count_of_condition
914292


###4.1 Question 2 : The top 10 conditions along with their frequency (note, that the Condition column can contain multiple conditions in each row, so you will need to separate these out and count each occurrence separately)

This SQL query retrieves the top 10 most common medical conditions found in the clinical_trials dataset. It works on the cleaned_conditions view, where each condition is already separated into individual rows. The query counts how often each condition appears, groups the results by condition, and then orders them from most to least frequent. The LIMIT 10 ensures that only the top 10 most frequent conditions are displayed.

In [0]:
%sql
SELECT 
    condition, COUNT(*) AS frequency
FROM 
    cleaned_conditions
GROUP BY 
    condition
ORDER BY 
    frequency DESC
    LIMIT 10;


condition,frequency
Healthy,10309
Breast Cancer,7941
Obesity,6954
Stroke,4484
Hypertension,4256
Depression,4196
Prostate Cancer,4071
Pain,4057
HIV Infections,3819
Cancer,3529


Databricks visualization. Run in Databricks to view.

##5.0 Question 3 : For studies with an end date, calculate the mean clinical trial length in months.

This SQL query calculates the average duration (in months) of clinical trials by finding the difference between the Start Date and Completion Date for each trial. It uses the months_between function to compute how long each trial lasted, filters out rows where either date is missing (NULL), and then takes the average of those durations. The result gives a general idea of how long clinical trials typically take to complete.

In [0]:
%sql
SELECT 
    AVG(months_between(`Completion Date`, `Start Date`)) AS avg_trial_length_months
FROM 
    cleaned_study_type
WHERE  
    `Completion Date` IS NOT NULL AND 
    `Start Date` IS NOT NULL;


avg_trial_length_months
35.571473612544345


###6.0 Question 4 : From the studies with a non-null completion date and a status of ‘Completed’ in the Study Status, calculate how many of these related to Diabetes each year.Display the trend over time in an appropriate visualisation. (For this you canassume all relevant studies will contain an exact match for ‘Diabetes’ or ‘diabetes’in the Conditions column.) 

This SQL code is analyzing how many diabetes-related clinical studies were completed each year. Here's a step-by-step explanation in simple, human-friendly terms:

Step 1: The code starts by breaking down the Conditions column (which may have multiple conditions separated by a pipe |) into individual entries using split and explode. It keeps only those studies that are marked as "COMPLETED" and have a valid Completion Date. It also extracts the year from each completion date.

Step 2: From this list, it filters the data to focus only on studies related to diabetes, ignoring case differences (like "Diabetes" or "diabetes").

Step 3: Finally, it counts how many diabetes-related studies were completed each year, and sorts the result in ascending order of year.

This helps us understand the trend of completed diabetes trials over the years, which can be useful for medical research, policy-making, or understanding focus areas in clinical studies.

In [0]:
%sql

-- Step 1: Spliting and explode the conditions
WITH exploded_conditions AS (
    SELECT 
        YEAR(`Completion Date`) AS year, 
        `Completion Date`,  -- Retaining Completion Date in this step
        `Study Status`,
        explode(split(`Conditions`, '\\|')) AS condition
    FROM 
        cleaned_study_type
    WHERE 
        UPPER(`Study Status`) = 'COMPLETED'  
        AND `Completion Date` IS NOT NULL
),

-- Step 2: Filtering the exploded conditions for diabetes-related studies
diabetes_studies AS (
    SELECT 
        year,
        `Completion Date`,
        condition
    FROM 
        exploded_conditions
    WHERE 
        LOWER(condition) = 'diabetes'  -- Case-insensitive match for 'Diabetes'
)

-- Step 3: Counting the number of diabetes-related studies per year
SELECT 
    year,
    COUNT(`Completion Date`) AS diabetes_studies_count
FROM 
    diabetes_studies
GROUP BY 
    year
ORDER BY 
    year ASC;


year,diabetes_studies_count
1995,1
1997,1
1998,3
1999,7
2000,5
2001,12
2002,16
2003,30
2004,41
2005,50


Databricks visualization. Run in Databricks to view.

###7.0 Conclusion


In this task, we conducted an in-depth analysis of the clinical trials dataset to uncover key insights. First, we examined the presence of missing or null values in critical columns like Study Type, Conditions, and Study Status, and addressed them by replacing null values with meaningful labels. We also split the Conditions column to isolate individual conditions and gain a better understanding of the most frequently studied medical conditions. Additionally, we calculated the average duration of clinical trials and tracked the number of diabetes-related studies completed each year. Through these steps, we were able to prepare the data effectively, revealing valuable trends and insights that lay the groundwork for further analysis or decision-making.